In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Celiac_Disease/GSE193442'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of human KIR+ CD8 T cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: PBMC'], 1: ['cell type: KIR+ CD8 T']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if the dataset contains gene expression data
is_gene_available = True  # Assuming gene expression data is available based on dataset title

# Check for the availability of variables in the sample characteristics
sample_characteristics = {
    0: ['tissue: PBMC'],
    1: ['cell type: KIR+ CD8 T']
}

# Determine trait_row
trait_row = None  # No indicator of 'Celiac_Disease' in the provided characteristics

# Determine age_row
age_row = None  # No indicator of 'age' in the provided characteristics

# Determine gender_row
gender_row = None  # No indicator of 'gender' in the provided characteristics

# Define conversion functions
def convert_trait(value):
    try:
        return 1 if 'Celiac_Disease' in value else 0
    except:
        return None

def convert_age(value):
    try:
        return float(value.split(':')[-1])
    except:
        return None

def convert_gender(value):
    try:
        gender = value.split(':')[-1].strip().lower()
        return 1 if gender == 'male' else 0 if gender == 'female' else None
    except:
        return None

# Save cohort information
save_cohort_info('GSE193442', './preprocessed/Celiac_Disease/cohort_info.json', is_gene_available, trait_row is not None)


A new JSON file was created at: ./preprocessed/Celiac_Disease/cohort_info.json
